In [ ]:
import keras
keras.__version__

In [ ]:
#users = ["alex", "ben", "miao", "natasha", "nick", "sarah", "sean", "spencer", "tim", "yijun"]
users = ["alex"]
#mats = ["cloth", "concrete", "door", "drywall", "laminant", "whiteboard"]
mats = ["cloth"]

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array, load_img

# grab all training file names, count total
files = []
for user in users:
    for mat in mats:
        images = [f for f in os.listdir("data/"+user+"/frames/") if mat in f]
        files.extend(images)
print "Frames found:", len(files)

# create single arrays to store data
X = np.ndarray(shape=(len(files), 256, 256), dtype=np.float32)
y = np.ndarray(shape=(len(files), 1), dtype=np.float32)

# load in all data from images into normalized array
i = 0
for user in users:
    print "  Loading data for", user
    for mat in mats:
        files = [f for f in os.listdir("data/"+user+"/frames/") if mat in f]
        for f in files:
            x = cv2.imread("data/"+user+"/frames/"+f, cv2.IMREAD_GRAYSCALE)
            x = (x - 128.0) / 128.0
            X[i] = x
            y[i] = float("hard" in f)
            i += 1
    print "    Loading", mat, "data complete"
print "All images converted to array!"

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7)
X_train = X_train.reshape((X_train.shape[0], 256, 256, 1))
X_test = X_test.reshape((X_test.shape[0], 256, 256, 1))

## Define CNN Architecture

In [ ]:
from keras import layers, models

# CNN: convolution and pooling
model = models.Sequential()
model.add(layers.Conv2D(8, (5, 5), activation='relu', padding='same', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(16, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# fully connected
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
batch_size = 100
epochs = 50

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
from keras.preprocessing import ImageDataGenerator as IDG
#IDG.apply_transform()
x

In [ ]:
x = 3

In [ ]:
x